# 1. LSTM 기본 모델

### - 1. 모델

In [5]:
# legacy not found error -> pip install torchtext==0.10.0 이후 재부팅

import dill
import time
import random
import numpy as np
from sklearn.metrics import roc_curve, auc

import nltk

nltk.download("punkt")
from nltk.tokenize import word_tokenize

import torch
import torch.nn as nn

from torchtext.legacy.data import Field
from torchtext.legacy.data import TabularDataset
from torchtext.legacy.data import BucketIterator
from torchtext.legacy.data import Iterator

[nltk_data] Downloading package punkt to /home/rkoh/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
RANDOM_SEED = 2020
torch.manual_seed(RANDOM_SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)

DATA_PATH = 'data/processed/'
#DATA_PATH = ''

### - 2. 모델 클래스 정의하기

In [7]:
class LSTMClassifier(nn.Module):
    def __init__(
        self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx
    ):
        super().__init__()
        self.embed_layer = nn.Embedding(
            # 생성할 Embedding Layer의 크기 정해주기
            # 보통 단어장 크기
            num_embeddings = num_embeddings,
            embedding_dim = embedding_dim,
            # 자연어 처리에서 배치별로 문장의 크기를 맞추기 위해서 짧은 문장에 Padding을 붙여서 길이를 맞춤
            # 특별한 의미는 없음
            # 학습에서 제외하기 위해 Padding이 단어장에서 어떤 숫자를 갖고 있는지 알려줌으로써 학습되지 않게 함
            padding_idx = pad_idx
        )
        self.lstm_layer = nn.LSTM(
            input_size = embedding_dim,
            hidden_size = hidden_size,
            num_layers = num_layers,
            bidirectional = True,
            dropout = 0.5
        )
        self.last_layer = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Dropout(0.5),
            nn.LeakyReLU(),
            # 가장 마지막 output 크기를 1로 줌
            nn.Linear(hidden_size, 1),
            # 확률로 변환시키기 위해 Sigmoid를 마지막 Activation으로 줌
            nn.Sigmoid()
        )

### - 3. 모델 파이프라인 정의

In [8]:
def forward(self, x):
    # 숫자로 이루어진 토큰을 Input으로 받는다고 가정
    # Input값을 Embedding 값으로 변환시켜주어야 함
    embed_x = self.embed_layer(x)
    # LSTM은 output, (Hidden State, Cell State)를 반환함
    # 이 중 State 값들은 사용하지 않으므로 반환받지 않음
    output, (_, _) = self.lstm_layer(embed_x)
    # LSTM의 output은 (배치 크기, 문장 길이, output size) size를 가짐
    # 가장 마지막 단어의 결괏값을 사용
    last_output = output[:, -1, :]
    # 문장의 마지막 단어의 output을 Fully Connected Layer에 통과시켜 확률값 계산
    last_output = self.last_layer(last_output)
    return last_output

# 2. 데이터셋 불러오기

### - 0. torchtext 사용 순서

In [9]:
# 파일에서 필요한 필드 선언
TEXT = Field(..)
LABEL = Field(..)
# 데이터 불러오기
dataset = TabularDataset(..)
# 불러온 데이터 단어장 만들기
TEXT.build_vocab(dataset)
# Data loader 만들기
data_loader = BucketIterator(dataset, ..)

SyntaxError: invalid syntax (1841777516.py, line 2)

### - 1. 문장 필드 정의

In [10]:
TEXT = Field(
    # 이 필드에는 문장이 들어온다는 것을 알려주는 True
    sequential = True,
    # 단어를 숫자로 변환시켜주는 단어장을 만들기 위해 이 필드 사용
    use_vocab = True,
    # 불러올 문장을 토크나이징할 함수 입력
    # word tokenize는 영어로 이루어진 문장을 토큰화시킬 때 가장 기본적으로 사용
    tokenize = word_tokenize,
    # 대소문자 구분.
    # lower = True : 모두 소문자 처리
    lower = True,
    # 자연어 처리 모듈별로 지원하는 데이터 형태
    # True : (배치, 문장)
    # False : (문장, 배치)
    batch_first = True
)

### - 2. 정답 필드 정의

In [11]:
LABEL = Field(
    # 문장 필드는 문장이 들어오기 때문에 True, 해당 열은 정답이 있으므로 False
    sequential = False,
    # 단어장을 생성하지 않음
    use_vocab = False,
    batch_first = True
)

### - 3. 데이터 불러오기

In [12]:
sat_train_data, sat_valid_data, sat_test_data = \
    TabularDataset.splits(
        # 데이터가 들어있는 폴더 경로   
        path = 'data/processed/',
        # 각각 train, val, test 파일명
        train = 'sat_train.tsv',
        validation = 'sat_valid.tsv',
        test = 'sat_test.tsv',
        # 데이터의 파일 포맷 형태. Tap Separated Value
        format = 'tsv',
        # 정의한 Field 입력
        # 실제 데이터 컬럼 순서로 입력해주어야 함
        # ('text', TEXT) : 이 데이터는 첫 번째 컬럼에 문장이 있고 그 컬럼명을 text로 하겠다
        fields = [('text', TEXT), ('label', LABEL)],
        # 데이터의 첫 번째 열에는 원래의 컬럼명이 들어 있음
        # 데이터로 사용되지 않기 때문에 따로 불러오지 않도록 해야 함
        # 1을 주어서 첫 번째 열 생략
        skip_header = 1        
    )
# 마지막으로 불러온 데이터 중 훈련 데이터를 이용해 TEXT 단어장 생성
# 그 중 2번 이상 나온 단어만 단어장에 사용
TEXT.build_vocab(sat_train_data, min_freq = 2)

### - 4. data loader 정의
- 문장의 길이를 보고 길이가 비슷한 문장끼리 묶음

In [13]:
sat_train_iterator, sat_valid_iterator, sat_test_iterator = \
    BucketIterator.splits(
        # 앞에서 불러온 데이터들을 묶어서 입력
        (sat_train_data, sat_valid_data, sat_test_data),
        # Data Loader에서 각 배치별 크기
        batch_size = 8,
        device = None,
        sort = False
    )

# 3. 학습
1. Data Loader에서 배치 불러오기
2. 배치를 모델에 넣어서 데이터 형태 맞추기
3. 배치를 모델에 넣어서 예측값 얻기
4. 정답과 예측값을 비교해서 Loss 계산하기
5. Loss를 이용해 모델 학습시키기

In [14]:
class LSTMClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super().__init__()
        self.embed_layer = nn.Embedding(
            num_embeddings=num_embeddings,
            embedding_dim=embedding_dim,
            padding_idx=pad_idx
        )
        self.lstm_layer = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_size,
            num_layers=num_layers,
            bidirectional=True,
            dropout=0.5
        )
        self.last_layer = nn.Sequential(
            nn.Linear(hidden_size * 2, hidden_size),
            nn.Dropout(0.5),
            nn.LeakyReLU(),
            nn.Linear(hidden_size, 1),
            nn.Sigmoid(),
        )

    def forward(self, x):
        embed_x = self.embed_layer(x)
        output, (_, _) = self.lstm_layer(embed_x)
        last_output = output[:, -1, :]
        last_output = self.last_layer(last_output)
        return last_output

### - 1. 모델 학습 함수 정의

# ---------------------------------------------------------------------------------

In [15]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'

def train(model, train_loader, optimizer, criterion, device):
    model.train()
    epoch_loss = 0
    # 입력 받은 Data Loader를 호출해 Batch를 부르는 코드
    for batch in train_loader:
        optimizer.zero_grad()
        # Batch는 두개의 Attribute를 갖고 있음 - fields = [('text', TEXT), ('label', LABEL)]
        # 'text'는 batch의 문장, 'label'은 Batch의 정답
        text = batch.text
        if text.shape[0] > 1:
            # 문장과 정답을 불러와서 필요한 데이터 형태로 변환
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            # 모델에 문장을 넣어서 결과 출력
            output = model(text).flatten()
            # 출력된 결과와 정답을 비교해서 Loss를 구함
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

### - 2. 모델 평가 함수 정의

In [16]:
def evaluate(model, valid_loader, criterion, device):
    # 평가를 위한 eval()
    model.eval()
    epoch_loss = 0
    # torch에서는 기본적으로 forward를 할 때 자동으로 gradient를 계산함
    # 평가할 때는 gradient 계산이 필요가 없음
    # 그래서 torch.no_grad()를 통해 gradient가 계산되지 않도록 함
    with torch.no_grad():
        for _, batch in enumerate(valid_loader):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)
            
            epoch_loss += loss.item()
            
    return epoch_loss / len(iterator)

# ---------------------------------------------------------------------------------

In [17]:
def train(model: nn.Module, iterator: Iterator, optimizer: torch.optim.Optimizer, criterion: nn.Module, device: str):
    model.train()
    epoch_loss = 0

    for _, batch in enumerate(iterator):
        optimizer.zero_grad()

        text = batch.text
        if text.shape[0] > 1:
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module, iterator: Iterator, criterion: nn.Module, device: str):
    model.eval()
    epoch_loss = 0

    with torch.no_grad():
        for _, batch in enumerate(iterator):
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            label = label.to(device)
            output = model(text).flatten()
            loss = criterion(output, label)
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def test(model: nn.Module, iterator: Iterator, device: str):
    model.eval()
    with torch.no_grad():
        y_real = []
        y_pred = []
        for batch in iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            output = model(text).flatten().cpu()
            y_real += [label]
            y_pred += [output]
        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)

    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)

    return auroc


def epoch_time(start_time: int, end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

### - 3. HyperParameter 선언

In [18]:
# Embedding Layer에 사용할 Padding Index를 가져옴
# TEXT.vocab.stoi : 앞서 만든 단어장에서 단어를 토큰으로 만들어주는 Dictionary
# TEXT.vocab.itos : 토큰을 단어로 바꿀 수 있음
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

# 학습시킬 모델 정의
lstm_classifier = LSTMClassifier(
    num_embeddings = len(TEXT.vocab),
    embedding_dim = 100,
    hidden_size = 200,
    num_layers = 4,
    pad_idx = PAD_IDX
)

# CPU - GPU 정의
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
_ = lstm_classifier.to(device)

# 가장 대중적으로 쓰이는 Optimizer - Adam
optimizer = torch.optim.Adam(lstm_classifier.parameters())
# 손실 함수 : Binary Cross Entropy 사용
bce_loss_fn = nn.BCELoss()

### - 4. 모델 학습

In [19]:
for epoch in range(N_EPOCHS):
    train_loss = train(
        lstm_classifier,
        # 학습
        sat_train_iterator,
        optimizer,
        bce_loss_fn,
        device
    )
    
    valid_loss = evaluate(
        lstm_classifier,
        # loss 계산
        sat_valid_iterator,
        bce_loss_fn,
        device
    )
    
    # 결과 출력
    print(f'Epoch : {epoch + 1:02}')
    print(f'\tTrain Loss : {train_loss : .5f}')
    print(f'\t Val. Loss : {valid_loss : .5f}')

Epoch : 01
	Train Loss :  0.52055
	 Val. Loss :  0.76535
Epoch : 02
	Train Loss :  0.51072
	 Val. Loss :  0.55905
Epoch : 03
	Train Loss :  0.47557
	 Val. Loss :  0.54456
Epoch : 04
	Train Loss :  0.43689
	 Val. Loss :  0.53841
Epoch : 05
	Train Loss :  0.42774
	 Val. Loss :  0.53824
Epoch : 06
	Train Loss :  0.40097
	 Val. Loss :  0.54642
Epoch : 07
	Train Loss :  0.44003
	 Val. Loss :  0.53912
Epoch : 08
	Train Loss :  0.42337
	 Val. Loss :  0.53085
Epoch : 09
	Train Loss :  0.42087
	 Val. Loss :  0.53794
Epoch : 10
	Train Loss :  0.43359
	 Val. Loss :  0.53596
Epoch : 11
	Train Loss :  0.43982
	 Val. Loss :  0.53834
Epoch : 12
	Train Loss :  0.46754
	 Val. Loss :  0.53222
Epoch : 13
	Train Loss :  0.42592
	 Val. Loss :  0.54775
Epoch : 14
	Train Loss :  0.42254
	 Val. Loss :  0.58045
Epoch : 15
	Train Loss :  0.41939
	 Val. Loss :  0.57019
Epoch : 16
	Train Loss :  0.42119
	 Val. Loss :  0.55278
Epoch : 17
	Train Loss :  0.43622
	 Val. Loss :  0.55147
Epoch : 18
	Train Loss :  0.439

### - 5. 모델저장

In [20]:
with open('baseline_model.dill', 'wb') as f:
    model = {
        'TEXT' : TEXT,
        'LABEL' : LABEL,
        'classifier' : lstm_classifier
    }
    dill.dump(model,f)

# 4. Test
- Area Under Receiver OperationCharacteristic(AUROC) 사용

### - 1. 테스트 함수 정의

In [21]:
def test(model, test_loader, device):
    model.eval()
    with torch.no_grad():
        y_real = []
        y_pred = []
        
        for batch in test_loader:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            text = text.to(device)
            
            output = model(text).flatten().cpu()
            
            # Test 결과를 보기 위해 각 Batch의 예측값을 list에 모음
            y_real += [label]
            y_pred += [output]
            
        # 모인 예측값들을 합침
        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)
    # 예측값과 정답을이용해 AUROC 계산
    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)
    
    return auroc

### - 2. 모델 성능 확인

In [22]:
_ = lstm_classifier.cpu()
test_auroc = test(
    # 학습이 끝난 모델 입력
    lstm_classifier,
    # 수능 test 데이터를 입력해 결과 확인
    sat_test_iterator,
    'cpu'
)

# 일반적으로 AUROC의 기준값은 0.5
print(f'SAT Dataset Test AUROC : {test_auroc:.5f}')

SAT Dataset Test AUROC : 0.84615


# 5. 성능 높이기 - 추가 데이터 이용

### - 1. 사전 학습 데이터 불러오기

In [23]:
TEXT = Field(
    sequential = True,
    use_vocab = True,
    tokenize = word_tokenize,
    lower = True,
    batch_first = True
)
Label = Field(
    sequential = False,
    use_vocab = False,
    batch_first = True
)

cola_train_data, cola_valid_data,cola_test_data = \
    TabularDataset.splits(
        path = DATA_PATH,
        train = 'cola_train.tsv',
        validation = 'cola_valid.tsv',
        test = 'cola_test.tsv',
        format = 'tsv',
        fields = [('text', TEXT), ('label', LABEL)],
        skip_header = 1
    )

# 사전학습은 사전학습 때 이용한 모델의 단어장을 유지하는 것이 중요함
# A모델과 B 모델에서의 단어는 같지만 토큰이 다르면 모델의 성능이 보장되지 않음
TEXT.build_vocab(cola_train_data, min_freq = 2)
cola_train_iterator, cola_valid_iterator, cola_test_iterator = \
    BucketIterator.splits(
        (cola_train_data, cola_valid_data, cola_test_data),
        batch_size = 32,
        device = None,
        sort = False
    )

### - 2. 추가 학습 데이터 불러오기

In [24]:
sat_train_data, sat_valid_data,sat_test_data = \
    TabularDataset.splits(
        path = DATA_PATH,
    train = "sat_train.tsv",
    validation = "sat_valid.tsv",
    test = "sat_test.tsv",
    format = "tsv",
    # CoLA데이터에서 만든 Field
    fields = [("text", TEXT), ("label", LABEL)],
    skip_header = 1
    )

sat_train_iterator, sat_valid_iterator,sat_test_iterator = \
    BucketIterator.splits(
        (sat_train_data, sat_valid_data, sat_test_data),
        batch_size = 8,
        device = None,
        sort = False
    )

### - 3. 모델 사전 학습

In [25]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

lstm_classifier = LSTMClassifier(
    num_embeddings = len(TEXT.vocab),
    embedding_dim = 100,
    hidden_size = 200,
    num_layers = 4,
    pad_idx = PAD_IDX
)
if torch.cuda.is_available():
    device = "cuda:0"
else:
    device = "cpu"
_ = lstm_classifier.to(device)

optimizer = torch.optim.Adam(lstm_classifier.parameters())
bce_loss_fn = nn.BCELoss()

for epoch in range(N_EPOCHS):
    train_loss = train(
        lstm_classifier,
        # CoLA 데이터를이용해 모델 학습
        cola_train_iterator,
        optimizer,
        bce_loss_fn,
        device
    )
    
    valid_loss = evaluate(
        lstm_classifier,
        cola_valid_iterator,
        bce_loss_fn,
        device    
    )
    
    print(f'Epoch : {epoch+1:02}')
    print(f'\tTrain Loss : {train_loss:.5f}')
    print(f'\t Val. Loss : {valid_loss:.5f}')
    


Epoch : 01
	Train Loss : 0.61572
	 Val. Loss : 0.61718
Epoch : 02
	Train Loss : 0.61236
	 Val. Loss : 0.61897
Epoch : 03
	Train Loss : 0.61123
	 Val. Loss : 0.61734
Epoch : 04
	Train Loss : 0.61139
	 Val. Loss : 0.61739
Epoch : 05
	Train Loss : 0.61087
	 Val. Loss : 0.61745
Epoch : 06
	Train Loss : 0.60869
	 Val. Loss : 0.61889
Epoch : 07
	Train Loss : 0.60907
	 Val. Loss : 0.61785
Epoch : 08
	Train Loss : 0.60964
	 Val. Loss : 0.61831
Epoch : 09
	Train Loss : 0.60958
	 Val. Loss : 0.62143
Epoch : 10
	Train Loss : 0.60851
	 Val. Loss : 0.61852
Epoch : 11
	Train Loss : 0.60938
	 Val. Loss : 0.61844
Epoch : 12
	Train Loss : 0.60842
	 Val. Loss : 0.62926
Epoch : 13
	Train Loss : 0.60918
	 Val. Loss : 0.61917
Epoch : 14
	Train Loss : 0.60811
	 Val. Loss : 0.61830
Epoch : 15
	Train Loss : 0.60980
	 Val. Loss : 0.61777
Epoch : 16
	Train Loss : 0.60883
	 Val. Loss : 0.61978
Epoch : 17
	Train Loss : 0.60843
	 Val. Loss : 0.61891
Epoch : 18
	Train Loss : 0.60803
	 Val. Loss : 0.61875
Epoch : 19

In [26]:
from copy import deepcopy

# 사전 학습한 모델과 추가 학습한 모델의 성능을 비교하기 위해 사전 학습한 모델 따로 저장
before_tuning_lstm_classifier = deepcopy(lstm_classifier)

### - 4. 모델 추가 학습

In [27]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

for epoch in range(N_EPOCHS):
    train_loss = train(
        lstm_classifier,
        sat_train_iterator,
        optimizer,
        bce_loss_fn,
        device
    )
    
    valid_loss = evaluate(
        lstm_classifier,
        sat_valid_iterator,
        bce_loss_fn,
        device
    )
    
    print(f'Epoch : {epoch+1:02}')
    print(f'\tTrain Loss : {train_loss:.5f}')
    print(f'\t Val. Loss : {valid_loss:.5f}')

Epoch : 01
	Train Loss : 0.48059
	 Val. Loss : 0.54516
Epoch : 02
	Train Loss : 0.44044
	 Val. Loss : 0.52474
Epoch : 03
	Train Loss : 0.43046
	 Val. Loss : 0.50973
Epoch : 04
	Train Loss : 0.42282
	 Val. Loss : 0.54304
Epoch : 05
	Train Loss : 0.43730
	 Val. Loss : 0.55573
Epoch : 06
	Train Loss : 0.41017
	 Val. Loss : 0.55850
Epoch : 07
	Train Loss : 0.42199
	 Val. Loss : 0.56403
Epoch : 08
	Train Loss : 0.41851
	 Val. Loss : 0.53660
Epoch : 09
	Train Loss : 0.39470
	 Val. Loss : 0.53152
Epoch : 10
	Train Loss : 0.41769
	 Val. Loss : 0.52412
Epoch : 11
	Train Loss : 0.43400
	 Val. Loss : 0.52917
Epoch : 12
	Train Loss : 0.43127
	 Val. Loss : 0.52669
Epoch : 13
	Train Loss : 0.42435
	 Val. Loss : 0.52834
Epoch : 14
	Train Loss : 0.43298
	 Val. Loss : 0.53807
Epoch : 15
	Train Loss : 0.41829
	 Val. Loss : 0.53283
Epoch : 16
	Train Loss : 0.41945
	 Val. Loss : 0.52609
Epoch : 17
	Train Loss : 0.42199
	 Val. Loss : 0.52492
Epoch : 18
	Train Loss : 0.42326
	 Val. Loss : 0.52838
Epoch : 19

### - 5. 모델 성능 비교

In [28]:
_ = before_tuning_lstm_classifier.cpu()
lstm_sat_test_auroc = test(
    before_tuning_lstm_classifier, sat_test_iterator, "cpu"
)

_ = lstm_classifier.cpu()
lstm_tuned_test_auroc = test(
    lstm_classifier, sat_test_iterator, "cpu"
)

print(f'Before fine-tuning SAT Dataset TEst AUROC : {lstm_sat_test_auroc:.5f}')
print(f'After fine-tuning SAT Dataset TEst AUROC : {lstm_tuned_test_auroc:.5f}')

Before fine-tuning SAT Dataset TEst AUROC : 0.76923
After fine-tuning SAT Dataset TEst AUROC : 0.65385


### - 6. 모델 저장

In [29]:
with open('before_tuning_model.dill', 'wb') as f:
    model = {
        'TEXT' : TEXT,
        'LABEL' : LABEL,
        'classifier' : before_tuning_lstm_classifier
    }
    dill.dump(model,f)
    
_ = lstm_classifier.cpu()
with open('after_tuning_model.dill', 'wb') as f:
    model = {
        'TEXT' : TEXT,
        'LABEL' : LABEL,
        'classifier' : lstm_classifier
    }
    dill.dump(model, f)

# 6. 심화 모델

### - 1. 모델 정의

In [30]:
class LSTMPoolingClassifier(nn.Module):
    def __init__(self, num_embeddings, embedding_dim, hidden_size, num_layers, pad_idx):
        super().__init__()
        self.embed_layer = nn.Embedding(
            num_embeddings = num_embeddings,
            embedding_dim = embedding_dim,
            padding_idx = pad_idx
        )
        self.lstm_layer = nn.LSTM(
            input_size = embedding_dim,
            hidden_size = hidden_size,
            num_layers = num_layers,
            bidirectional = True,
            dropout = 0.5,
            batch_first = True
        )
        self.last_layer = nn.Sequential(
            nn.Linear(2 * hidden_size, 1),
            nn.Dropout(p = 0.5),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        # Token 으로 들어온 데이터를 Emedding하여 값으로 변환
        x = self.embed_layer(x)
        # 변환된 값을 LSTM에 넣음
        output, _ = self.lstm_layer(x)
        # LSTM의 결과를 Max Pooling
        pool = nn.functional.max_pool1d(output.transpose(1, 2), x.shape[1])
        # Max Pooling 결과를 Fully Connected Layer에 넣기 위해 Shape를 맞춤
        pool = pool.transpose(1, 2).squeeze()
        # Fully Connected Layer에 넣어서 결과반환
        output = self.last_layer(pool)
        
        return output.squeeze()

### - 2. 모델 사전 학습

In [31]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

lstm_pool_classifier = LSTMPoolingClassifier(
    num_embeddings = len(TEXT.vocab),
    embedding_dim = 100,
    hidden_size = 200,
    num_layers = 4,
    pad_idx = PAD_IDX
)

if torch.cuda.is_available():
    device = 'cuda:0'
else:
    device = 'cpu'
_ = lstm_pool_classifier.to(device)

optimizer = torch.optim.Adam(lstm_pool_classifier.parameters())
bce_loss_fn = nn.BCELoss()

for epoch in range(N_EPOCHS):
    train_loss = train(
        lstm_pool_classifier,
        # 모델을 CoLA 데이를 이용해 학습
        cola_train_iterator,
        optimizer,
        bce_loss_fn,
        device
    )
    valid_loss = evaluate(
        lstm_pool_classifier,
        cola_valid_iterator,
        bce_loss_fn,
        device
    )
    
    print(f'Epoch : {epoch+1:02}')
    print(f'\tTrain Loss : {train_loss:.5f}')
    print(f'\t Val. Loss : {valid_loss:.5f}')

/home/rkoh/anaconda3/envs/vip/lib/python3.8/site-packages/torch/nn/functional.py:652: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool1d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch : 01
	Train Loss : 0.65178
	 Val. Loss : 0.62461
Epoch : 02
	Train Loss : 0.65243
	 Val. Loss : 0.62057
Epoch : 03
	Train Loss : 0.64198
	 Val. Loss : 0.63551
Epoch : 04
	Train Loss : 0.64251
	 Val. Loss : 0.62867
Epoch : 05
	Train Loss : 0.63104
	 Val. Loss : 0.61683
Epoch : 06
	Train Loss : 0.62215
	 Val. Loss : 0.62509
Epoch : 07
	Train Loss : 0.60219
	 Val. Loss : 0.61948
Epoch : 08
	Train Loss : 0.59549
	 Val. Loss : 0.62963
Epoch : 09
	Train Loss : 0.57439
	 Val. Loss : 0.63638
Epoch : 10
	Train Loss : 0.56506
	 Val. Loss : 0.63001
Epoch : 11
	Train Loss : 0.54868
	 Val. Loss : 0.63740
Epoch : 12
	Train Loss : 0.53263
	 Val. Loss : 0.64127
Epoch : 13
	Train Loss : 0.51682
	 Val. Loss : 0.67401
Epoch : 14
	Train Loss : 0.49689
	 Val. Loss : 0.66480
Epoch : 15
	Train Loss : 0.48611
	 Val. Loss : 0.68894
Epoch : 16
	Train Loss : 0.47420
	 Val. Loss : 0.69010
Epoch : 17
	Train Loss : 0.45635
	 Val. Loss : 0.67690
Epoch : 18
	Train Loss : 0.46026
	 Val. Loss : 0.67834
Epoch : 19

In [32]:
# 성능 비굘르 위해 모델 따로 저장
before_tuning_lstm_pool_classifier = deepcopy(lstm_pool_classifier)

### - 3. 모델 추가 학습

In [33]:
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]
N_EPOCHS = 20

for epoch in range(N_EPOCHS):
    train_loss = train(
        # 앞서 학습한 모델 입력
        lstm_pool_classifier,
        # 수능 데이터 사용
        sat_train_iterator,
        optimizer,
        bce_loss_fn,
        device
    )
    valid_loss = evaluate(
        lstm_pool_classifier,
        sat_valid_iterator,
        bce_loss_fn,
        device
    )
    
    print(f'Epoch : {epoch+1:02}')
    print(f'\tTrain Loss : {train_loss:.5f}')
    print(f'\t Val. Loss : {valid_loss:.5f}')

Epoch : 01
	Train Loss : 0.76298
	 Val. Loss : 0.70000
Epoch : 02
	Train Loss : 0.60446
	 Val. Loss : 0.61781
Epoch : 03
	Train Loss : 0.47171
	 Val. Loss : 0.62794
Epoch : 04
	Train Loss : 0.46750
	 Val. Loss : 0.63935
Epoch : 05
	Train Loss : 0.39454
	 Val. Loss : 0.75749
Epoch : 06
	Train Loss : 0.43597
	 Val. Loss : 0.60489
Epoch : 07
	Train Loss : 0.39991
	 Val. Loss : 0.61854
Epoch : 08
	Train Loss : 0.46557
	 Val. Loss : 0.64095
Epoch : 09
	Train Loss : 0.44545
	 Val. Loss : 0.65888
Epoch : 10
	Train Loss : 0.41117
	 Val. Loss : 0.68508
Epoch : 11
	Train Loss : 0.39435
	 Val. Loss : 0.71774
Epoch : 12
	Train Loss : 0.41486
	 Val. Loss : 0.76167
Epoch : 13
	Train Loss : 0.39266
	 Val. Loss : 0.75686
Epoch : 14
	Train Loss : 0.41874
	 Val. Loss : 0.77138
Epoch : 15
	Train Loss : 0.36933
	 Val. Loss : 0.80601
Epoch : 16
	Train Loss : 0.30590
	 Val. Loss : 0.84819
Epoch : 17
	Train Loss : 0.40920
	 Val. Loss : 0.89918
Epoch : 18
	Train Loss : 0.39506
	 Val. Loss : 0.92480
Epoch : 19

### - 4. 성능 비교

In [34]:
_ = before_tuning_lstm_pool_classifier.cpu()
_ = lstm_pool_classifier.cpu()

pool_sat_test_auroc = test(before_tuning_lstm_pool_classifier, sat_test_iterator, 'cpu')
pool_tuned_test_auroc = test(lstm_pool_classifier, sat_test_iterator, 'cpu')

print(f'Before fine-tuning SAT Dataset Test AUROC : {pool_sat_test_auroc:.5f}')
print(f'After fine-tuning SAT Dataset Test AUROC : {pool_tuned_test_auroc:.5f}')

Before fine-tuning SAT Dataset Test AUROC : 0.38462
After fine-tuning SAT Dataset Test AUROC : 0.23077


### - 5. 모델 저장

In [35]:
with open('advanced_before_tuning_model.dill', 'wb') as f:
    model = {
        'TEXT' : TEXT,
        'LABEL' : LABEL,
        'classifier' : before_tuning_lstm_pool_classifier
    }
    dill.dump(model, f)
    
with open('advanced_after_tuning_model.dill', 'wb') as f:
    model = {
        'TEXT' : TEXT,
        'LABEL' : LABEL,
        'classifier' : lstm_pool_classifier
    }
    dill.dump(model, f)

# 6. 데모

### - 1. 성능 비교 함수 정의

In [40]:
def test(model_path):
    # 주어진 파일 이름으로 저장한 모델 불러오기
    with open(model_path, 'rb') as f:
        model = dill.load(f)
        
    # 수능 Test 데이터셋. Test만 불러오기
    # path는 폴더 경로 대신 불러올 파일명 입력
    sat_test_data = TabularDataset(
        path = f'{DATA_PATH}/sat_test.tsv',
        format = 'tsv',
        # 각 모델별로 정의한 Field 사용
        fields = [
            ('text', model['TEXT']),
            ('label', model['LABEL'])
        ],
        skip_header = 1
    )
    
    # 불러온 데이터로 Data Loader 생성
    sat_test_iterator = BucketIterator(
        sat_test_data,
        batch_size = 8,
        device = None,
        sort = False,
        shuffle = False
    )
    # 저장한 모델 불러오기
    classifier = model['classifier']
    
    with torch.no_grad():
        y_real = []
        y_pred = []
        classifier.eval()
        for batch in sat_test_iterator:
            text = batch.text
            label = batch.label.type(torch.FloatTensor)
            
            output = classifier(text).flatten().cpu()
            
            y_real += [label]
            y_pred += [output]
            
        y_real = torch.cat(y_real)
        y_pred = torch.cat(y_pred)
        
    fpr, tpr, _ = roc_curve(y_real, y_pred)
    auroc = auc(fpr, tpr)
    
    return auroc.round(5)
    

### - 2. 성능 비교

In [41]:
# 모델들 저장한 파일명
model_list = [
    'baseline_model.dill',
    'before_tuning_model.dill',
    'after_tuning_model.dill',
    'advanced_before_tuning_model.dill',
    'advanced_after_tuning_model.dill'
]

test_auroc = []
for file_name in model_list:
    # 파일 이름의 .dill 지우고 사용
    model_name = file_name.replace(".dill", "")
    # 앞에서 정의한 test 함수 사용
    auroc = test(file_name)
    # 모델의 성능이 좋은 순서대로 정렬
    test_auroc += [(model_name, auroc)]
    
# 5
test_auroc = sorted(test_auroc, key = lambda x: x[1], reerse = True)
for rank, (model_name, auroc) in enumerate(test_auroc):
    print(f'Rank {rank+1} - {model_name:30} - Test AUROC : {auroc:.5f}')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu! (when checking arugment for argument index in method wrapper_index_select)

### - 3. 문제 풀이 함수 정의

In [42]:
def predict_problem(model_path, problem):
    with open(model_path, 'rb') as f:
        model = dill.load(f)
    TEXT = model['TEXT']
    classifier = model['classifier']
    
    # 입력받은 문장에서 필요없는 기호 지우기
    problem = list(map(lambda x: x.replace('[', '').replace(']', ''), problem))
    
    # 문장을 단어로 나눠줌
    tokenized_sentences = [word_tokenize(sentence) for sentence in problem]
    sentences = []
    for tokenized_sentence in tokenized_sentences:
        # 단어를 TEXT에 들어 있는 단어장을 이용해 토큰으로 변환
        sentences.append([TEXT.vocab.stoi[word] for word in tokenized_sentence])
        
    with torch.no_grad():
        classifier.eval()
        predict = []
        for sentence in sentences:
            sentence = torch.LongTensor([sentence])
            # 모델에 넣어 결과를 출력하고 저장
            predict += [classifier(sentence).item()]
            
    return predict

### - 4. 여러 모델을 처리하는 함수 정의

In [ ]:
def predict_problem_with_models(model_list, problem):
    scores = {}
    for file_name in model_list:
        model_name = file_name.replace('.dill', '')
        # 각 모델별로 predict_problem을 통해서 결과 출력
        score = predict_problem(file_name, problem)
        scores[model_name] = score
        
    score_df = pd.DataFrame(scores).T
    score_df.columns = [f'answer_{i}_score' for i in range(1, 6)]
    
    selected_answer = pd.Series(
        # 위에서 도출된 Score중 가장 작은 값이 있는 위치를 찾음 : 0~4
        # 문제의 보기와 숫자를 맞추기 위해 +1
        np.argmin(score_df.values, 1) + 1
        index = score_df.index,
        name = "selected_answer"
    )
    
    return pd.concat([selected_answer, score_df], 1)